# Runbook

## Goals

- Archecturual overview
- Overview of Kubernetes Tools (k14s)
- Setup and deployment of CF on a Kubernetes cluster
- Deploy a sample app and how an application image is built
- Review the changes made to K8S

## Deploy CF... Explain Later

Connect to an existing cluster named tas4k8s. Rename an existing cluster if you have to.

In [ ]:
kubectl config use-context tas4k8s

### K14S Toolchain

https://k14s.io/

In [ ]:
ll /usr/local/bin | grep -i 'kapp\|kbld\|ytt'

### BOSH CLI

In [ ]:
ll /usr/local/bin | grep -i 'bosh'

### TAS App Bundle

https://network.pivotal.io/products/tas-for-kubernetes

In [ ]:
pivnet login --api-token $PIVNET_TOKEN

In [ ]:
pivnet download-product-files --product-slug='tas-for-kubernetes' --release-version='0.1.0' --product-file-id=660279

In [ ]:
mkdir -p tas

In [ ]:
tar -xf tanzu-application-service.0.1.0-build.252.tar --directory tas

### Configuration

In [ ]:
cat tas/bin/generate-values.sh

In [ ]:
cat tas/config/cf-for-k8s/hack/generate-values.sh | grep -i 'bosh'

In [ ]:
cat tas/hack/generate-values.sh | grep -i 'bosh'

In [ ]:
mv tas/custom-overlays/replace-loadbalancer-with-clusterip.yaml tas/config-optional

In [ ]:
wget -P tas/custom-overlays https://raw.githubusercontent.com/cloudfoundry/cf-for-k8s/master/config-optional/remove-resource-requirements.yml

In [ ]:
mkdir -p configuration-values

In [ ]:
tas/bin/generate-values.sh -d "$TAS4K8S_DOMAIN" > configuration-values/deployment-values.yml

In [ ]:
cat configuration-values/deployment-values.yml

In [ ]:
echo "system_registry:" >> configuration-values/deployment-values.yml
echo "  hostname: registry.pivotal.io" >> configuration-values/deployment-values.yml
echo "  username: $PIV_REGISTRY_USERNAME" >> configuration-values/deployment-values.yml
echo "  password: $PIV_REGISTRY_PASSWORD" >> configuration-values/deployment-values.yml

In [ ]:
echo "app_registry:" >> configuration-values/deployment-values.yml
echo "  hostname: $APP_REGISTRY_HOSTNAME" >> configuration-values/deployment-values.yml
echo "  repository: $APP_REGISTRY_REPOSITORY" >> configuration-values/deployment-values.yml
echo "  username: $APP_REGISTRY_USERNAME" >> configuration-values/deployment-values.yml
echo "  password: $APP_REGISTRY_PASSWORD" >> configuration-values/deployment-values.yml

### Install

Run in a separate terminal window due to time and output length.

`tas/bin/install-tas.sh configuration-values`

## Architecture Overview

![components](./imgs/1-components.jpg)

![integration](./imgs/2-integration.jpg)

![pathway-to-ga](./imgs/3-pathway-to-ga.jpg)

## Update DNS

In [ ]:
kubectl -n istio-system get svc istio-ingressgateway

## Login & Verify

`cf login -a https://api.$TAS4K8S_DOMAIN --skip-ssl-validation -u admin -p "[password]"`

In [ ]:
cf enable-feature-flag diego_docker

In [ ]:
cf create-org tas4k8s

In [ ]:
cf create-space -o tas4k8s demo

In [ ]:
cf target -o tas4k8s -s demo

### Push Docker Image

In [ ]:
cf push spring-music --hostname spring-music --docker-image bhaktapk/spring-music

### Push An App

In [ ]:
git clone https://github.com/cloudfoundry-samples/cf-sample-app-nodejs.git

In [ ]:
cf push nodejs -p cf-sample-app-nodejs -f cf-sample-app-nodejs/manifest.yml --hostname nodejs

In [ ]:
cf logs nodejs --recent

In [ ]:
cf scale nodejs -i 2

In [ ]:
cf app nodejs

## Kubernetes Deep Dive

In [ ]:
kubectl get namespaces

In [ ]:
kubectl -n build-service get deploy,svc

In [ ]:
kubectl -n cf-blobstore get deploy,svc

In [ ]:
kubectl -n cf-db get statefulset,svc

In [ ]:
kubectl -n cf-system get deploy,svc

In [ ]:
kubectl -n cf-workloads get statefulset,svc -o wide

In [ ]:
kubectl -n cf-workloads-staging get all

In [ ]:
kubectl -n istio-system get daemonset,deploy,hpa,svc

In [ ]:
kubectl -n kpack get deploy,svc

In [ ]:
kubectl -n metacontroller describe statefulset metacontroller

## Future

![initial-ga](imgs/4-initial-ga.jpg)

![roadmap](imgs/5-roadmap.jpg)

## Cleanup

In [ ]:
kapp delete -a cf

In [ ]:
rm -rf cf-sample-app-nodejs

In [ ]:
rm -rf configuration-values

In [ ]:
rm -rf tas

In [ ]:
rm -rf tanzu-application-service.0.1.0-build.252.tar